In [34]:
import pandas as pd
df = pd.read_csv(r'G:\HashCode_23\PLID_Bookings_Problem1.csv')

In [35]:
df.head()

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date
0,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2022,AUG FY2022,204,01-08-2016
1,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2022,OCT FY2022,564,01-10-2016
2,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2022,SEP FY2022,1476,01-09-2016
3,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2002,AUG FY2002,374,01-08-2017
4,CSWBU,CBR8,CBR-D121-DS-MOD,Q1 FY2002,OCT FY2002,6094,01-10-2017


In [36]:
bu_types = df["Business Unit"].unique()
pf_types = df["Product Family"].unique()
plid_types = df["PLID"].unique()
fq_types = df["Fiscal Quarter"].unique()
fm_types = df["Fiscal Month"].unique()

In [37]:
months = {'JAN':1, 'FEB':2, 'MAR':3, 'APR':4, 'MAY':5, 'JUN':6, 'JUL':7, 'AUG':8, 'SEP':9, 'OCT':10, 'NOV':11, 'DEC':12}

In [38]:
df["Fiscal Month"] = df["Fiscal Month"].apply(lambda x: f'{months[x[0:3]]}-{int("20"+x[-2]+x[-1])}')

In [39]:
df["Fiscal Month"] = pd.to_datetime(df["Fiscal Month"], format="%m-%Y")

In [40]:
df['Fiscal Quarter'] = df['Fiscal Month'].dt.quarter
df['Fiscal Year'] = df['Fiscal Month'].dt.year

In [41]:
df["mean_across_quarter"] = df.groupby(['PLID', 'Fiscal Quarter', 'Fiscal Year'], as_index=False).agg({'Booked_Qty': 'mean'})['Booked_Qty']

In [42]:
df.head()

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,Fiscal Year,mean_across_quarter
0,CSWBU,CBR8,CBR-D121-DS-MOD,3,2022-08-01,204,01-08-2016,2022,8.666667
1,CSWBU,CBR8,CBR-D121-DS-MOD,4,2022-10-01,564,01-10-2016,2022,505.000000
2,CSWBU,CBR8,CBR-D121-DS-MOD,3,2022-09-01,1476,01-09-2016,2022,12.333333
3,CSWBU,CBR8,CBR-D121-DS-MOD,3,2002-08-01,374,01-08-2017,2002,73.500000
4,CSWBU,CBR8,CBR-D121-DS-MOD,4,2002-10-01,6094,01-10-2017,2002,193.500000


In [43]:
df['Booked_Qty_Change'] = df.groupby(['PLID', 'Fiscal Quarter', 'Fiscal Year'])['mean_across_quarter'].pct_change()

In [44]:
df["mean_across_quarter"] = df["mean_across_quarter"].fillna(0)
df["Booked_Qty_Change"] = df["Booked_Qty_Change"].fillna(0)

In [46]:
feature_df = df[["mean_across_quarter","Booked_Qty_Change"]]

In [47]:
n_array = feature_df.to_numpy(dtype = 'float64')

In [51]:
import numpy as np

In [52]:
inf_array = np.isinf(n_array)

In [53]:
n_array = np.where(inf_array, 0, n_array)

In [54]:
feature_df = pd.DataFrame(n_array, columns = ["mean_across_quarter","Booked_Qty_Change"])

In [55]:
df["mean_across_quarter"] = feature_df["mean_across_quarter"]
df["Booked_Qty_Change"] = feature_df["Booked_Qty_Change"]

In [56]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [57]:
from sklearn.ensemble import RandomForestRegressor

# Train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=1000, random_state=42)
features = ["mean_across_quarter", 'Booked_Qty_Change']

In [58]:
rf_model.fit(train_data[features], train_data['Booked_Qty'])

RandomForestRegressor(n_estimators=1000, random_state=42)

In [60]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Evaluate the performance of the model on the test data
test_predictions = rf_model.predict(test_data[features])
mae = mean_absolute_error(test_data['Booked_Qty'], test_predictions)
rmse = mean_squared_error(test_data['Booked_Qty'], test_predictions, squared=False)
print('MAE: {:.2f}'.format(mae))
print('RMSE: {:.2f}'.format(rmse))

MAE: 9131.52
RMSE: 18112.34
